In [3]:
import pandas as pd
import os
import shutil
import cv2
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# === USER SETTINGS ===
CSV_PATH = "annotations.csv"
IMAGES_DIR = "/path/to/images"  # folder with all images
OUTPUT_DIR = "yolo_dataset"     # output folder
SELECTED_CLASSES = ["Buccinid snail", "Spider crab", "Polynoid worms", "Zoarcid fish"]
RANDOM_SEED = 42
VAL_RATIO = 0.2

# === SETUP ===
os.makedirs(OUTPUT_DIR, exist_ok=True)
for split in ["train", "val"]:
    os.makedirs(f"{OUTPUT_DIR}/images/{split}", exist_ok=True)
    os.makedirs(f"{OUTPUT_DIR}/labels/{split}", exist_ok=True)

In [ ]:
df = pd.read_csv(CSV_PATH)

# Check first rows
df.head()

In [ ]:
# Unify class labels (optional)
df["name_sp"] = df["name_sp"].replace({"Polynoid worm": "Polynoid worms"})

# Count total annotations per class
total_counts = df["name_sp"].value_counts()

# Count valid annotations (non-NaN x1/y1/x2/y2)
valid_df = df[df[["x1", "y1", "x2", "y2"]].notnull()]
valid_counts = valid_df["name_sp"].value_counts()

# Count unique images per class (from valid entries)
image_counts = valid_df.groupby("name_sp")["name_img"].nunique()

# Combine all
summary = pd.DataFrame({
    "Total_Annotations": total_counts,
    "Valid_Annotations": valid_counts,
    "Valid_Images": image_counts
}).fillna(0).astype(int)

summary = summary.sort_values("Valid_Annotations", ascending=False)
display(summary)


In [ ]:
# Unify class labels (optional)
df["name_sp"] = df["name_sp"].replace({"Polynoid worm": "Polynoid worms"})

# Count total annotations per class
total_counts = df["name_sp"].value_counts()

# Count valid annotations (non-NaN x1/y1/x2/y2)
valid_df = df[df[["x1", "y1", "x2", "y2"]].notnull()]
valid_counts = valid_df["name_sp"].value_counts()

# Count unique images per class (from valid entries)
image_counts = valid_df.groupby("name_sp")["name_img"].nunique()

# Combine all
summary = pd.DataFrame({
    "Total_Annotations": total_counts,
    "Valid_Annotations": valid_counts,
    "Valid_Images": image_counts
}).fillna(0).astype(int)

summary = summary.sort_values("Valid_Annotations", ascending=False)
display(summary)